In [5]:
import time 
import re
import ctypes
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import  expected_conditions as EC 
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException

# ctypes.windll.kernal32.SetThreadExecutionState(x80000002) # stops sleep (windows)


In [ ]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

driver.get('https://www.laliga.com/en-GB/laliga-easports/clubs')

In [ ]:
retry_attempts = 0
max_retries = 1
clubs = []
collected_data = []
processed_clubs = set()

while retry_attempts <= max_retries:
    try:

        clubs.clear()


        wait = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'styled__TextStyled-sc-1mby3k1-0 kYCCIm'))
        )
        club_names_elements = driver.find_elements(By.CLASS_NAME, 'styled__TextStyled-sc-1mby3k1-0 kYCCIm')
        for element in club_names_elements:
            clubs.append(element.text)

        print("Club names extracted:", clubs)


        wait = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'link'))
        ) 
        club_links = driver.find_elements(By.CLASS_NAME, 'link')
        

        for element in club_links:  
            club_name = element.text

            if club_name in processed_clubs:
                print(f"Skipping {club_name} - already processed")
                continue


            element.click()
            print(f"Clicked link for {club_name}")
            

            wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'styled__TextStyled-sc-1mby3k1-0')))
            wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'styled__TextStyled-sc-1mby3k1-0 dUHLTw')))
            wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'styled__TextStyled-sc-1mby3k1-0 bDmhbs')))

            club_name_extracted = driver.find_element(By.CLASS_NAME, 'styled__TextStyled-sc-1mby3k1-0').text
            player_name = driver.find_element(By.CLASS_NAME, 'styled__TextStyled-sc-1mby3k1-0 dUHLTw').text
            player_position = driver.find_element(By.CLASS_NAME, 'styled__TextStyled-sc-1mby3k1-0 bDmhbs').text

            club_info = {
                "Club Name": club_name_extracted, 
                "Player Name": player_name,
                "Player Position": player_position
            }
            collected_data.append(club_info)

            print(f"Collected data for {club_name_extracted}: {club_info}")
            processed_clubs.add(club_name)


            driver.back()
            wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'link')))
            print("Returned to the main page.")
            break

        break

    except TimeoutException:
        retry_attempts += 1
        print(f"Timeout exception occurred. Attempt {retry_attempts} of {max_retries + 1}")
        if retry_attempts > max_retries:
            print("Max retries reached. Stopping script")

if not clubs:
    print("No club names were found")
elif not collected_data:
    print("No data was collected. There may have been an issue.")
else:
    print("Data collection completed successfully.")
